In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
def preclear(df):
    df['Sex'] = df['Sex'].map({'female':1, 'male':0})
    meanAge = df.Age.mean()
    df['Age'] = df['Age'].fillna(value=meanAge)
    meanFare = df.Fare.mean()
    df['Fare'] = df['Fare'].fillna(value=meanFare)
    return df

In [7]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/gender_submission.csv')

train = preclear(train)
test = preclear(test)
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [8]:
train.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,0.477990,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
train.groupby(['Sex'])['Survived', 'Fare', 'Age', 'SibSp'].mean()

,Survived,Fare,Age,SibSp
Sex,,,,
0,0.188908,25.523893,30.505824,0.429809
1,0.742038,44.479818,28.216730,0.694268


In [11]:
valid_labels = [
    'Pclass', 
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare'
];

X = train[valid_labels]
y = train.Survived

X_test = test[valid_labels]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass    891 non-null int64
Sex       891 non-null int64
Age       891 non-null float64
SibSp     891 non-null int64
Parch     891 non-null int64
Fare      891 non-null float64
dtypes: float64(2), int64(4)
memory usage: 41.8 KB


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.333, random_state=42)


In [13]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(
    solver='lbfgs',
    alpha=1e-5,
    hidden_layer_sizes=(10,10),
    random_state=1
)
clf.fit(X_train, y_train)
y_proba = clf.predict_proba(X_cv)[:,1]
y_pred = clf.predict(X_cv)

s_pred = clf.predict(X_test)

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(
    random_state=1,
    C=0.15
)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_cv)

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y_cv, y_pred)
#accuracy_score(submission['Survived'], s_pred)
#s_pred.to_csv()


0.81818181818181823

In [16]:
columns = ['PassengerId', 'Survived']
index = np.arange(s_pred.shape[0]) # array of numbers for the number of samples
import pandas
df = pandas.DataFrame(columns=columns, index = index)

for idx in range(s_pred.shape[0]):
    df.ix[idx] = [submission['PassengerId'][idx], s_pred[idx]]

df.to_csv('sub.csv', index=False)